In [1]:
import numpy as np
import sbi_tvb
from matplotlib import rcParams
from sbi_tvb.inference import TvbInference
from sbi_tvb.prior import Prior
from tvb.simulator.lab import *

/Users/pipeline/anaconda/anaconda3/envs/tvb-inversion/lib/python3.8/site-packages/tqdm-4.64.0-py3.8.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2022-05-13 15:41:59,981 - INFO - tvb.storage.h5.encryption.data_encryption_handler - Cannot import syncrypto library.


In [2]:
rcParams['figure.figsize'] = 15, 6


In [3]:
#
# Simulation setup
#
dt = 0.005
nsigma = 0.035
seed = 42
sim_len = 30e3
G = 2.45
BOLD_TR = 2250

In [4]:
# Define connectivity
sbi_tvb_path = os.path.dirname(os.path.dirname(sbi_tvb.__file__))
weights = np.loadtxt(os.path.join(sbi_tvb_path, 'data_input_files', 'SC_Schaefer7NW100p_nolog10.txt'))

In [5]:
def build_simulator():
    sbi_tvb_path = os.path.dirname(os.path.dirname(sbi_tvb.__file__))
    weights = np.loadtxt(os.path.join(sbi_tvb_path, 'data_input_files', 'SC_Schaefer7NW100p_nolog10.txt'))

    model = models.MontbrioPazoRoxin(
        eta=np.r_[-4.6],
        J=np.r_[14.5],
        Delta=np.r_[0.7],
        tau=np.r_[1],
    )

    magic_number = 124538.470647693
    weights_orig = weights / magic_number
    conn = connectivity.Connectivity(
        weights=weights_orig,
        region_labels=np.array(np.zeros(np.shape(weights_orig)[0]), dtype='<U128'),
        tract_lengths=np.zeros(np.shape(weights_orig)),
        areas=np.zeros(np.shape(weights_orig)[0]),
        speed=np.array(np.Inf, dtype=float),
        centres=np.zeros(np.shape(weights_orig)[0]))  # default 76 regions

    integrator = integrators.HeunStochastic(
        dt=0.005,
        noise=noise.Additive(
            nsig=np.r_[0.035, 0.035 * 2],
            noise_seed=42
        )
    )

    _monitors = [monitors.TemporalAverage(period=0.1)]

    cond_speed = np.Inf

    sim = simulator.Simulator(model=model,
                              connectivity=conn,
                              coupling=coupling.Scaling(
                                  a=np.r_[2.45]
                              ),
                              conduction_speed=cond_speed,
                              integrator=integrator,
                              monitors=_monitors,
                              simulation_length=30e3
                              )
    return sim


In [6]:
sim = build_simulator()
print("Build TvbInference object")
tvb_inference = TvbInference(sim=sim, priors=[Prior('coupling.a', 1.5, 3.2)])

Build TvbInference object


In [7]:
print("Sample priors")
tvb_inference.sample_priors(num_simulations=3, num_workers=1)

Sample priors
   INFO  White noise configured with dt=0.005
FC_Corr
   INFO  White noise configured with dt=0.005
FC_Corr


Running 3 simulations.:   0%|                                                                                                   | 0/3 [00:00<?, ?it/s]

Using params: [1.7929152]
   INFO  White noise configured with dt=0.005


Running 3 simulations.:  33%|██████████████████████████████                                                            | 1/3 [02:49<05:39, 169.73s/it]

FC_Corr
Using params: [1.7582259]
   INFO  White noise configured with dt=0.005


Running 3 simulations.:  67%|████████████████████████████████████████████████████████████                              | 2/3 [05:38<02:49, 169.32s/it]

FC_Corr
Using params: [1.6520705]
   INFO  White noise configured with dt=0.005


Running 3 simulations.: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 3/3 [08:25<00:00, 168.40s/it]

FC_Corr


(tensor([[1.7929],
         [1.7582],
         [1.6521]]),
 tensor([[0.1127, 0.1126, 0.1149,  ..., 0.0647, 0.1318, 0.0016],
         [0.1127, 0.1135, 0.1148,  ..., 0.0630, 0.1327, 0.0021],
         [0.1128, 0.1125, 0.1147,  ..., 0.0608, 0.1317, 0.0018]]))

In [8]:
print("Train")
tvb_inference.train()

Train
   INFO  White noise configured with dt=0.005
FC_Corr
   INFO  White noise configured with dt=0.005
FC_Corr


/Users/pipeline/anaconda/anaconda3/envs/tvb-inversion/lib/python3.8/site-packages/sbi/neural_nets/flow.py:108: UserWarning: In one-dimensional output space, this flow is limited to Gaussians
  warn(f"In one-dimensional output space, this flow is limited to Gaussians")


Neural network successfully converged after 37 epochs.


DirectPosterior(
               method_family=snpe,
               net=<a Flow, see `.net` for details>,
               prior=BoxUniform(Uniform(low: tensor([1.5000]), high: tensor([3.2000])), 1),
               x_shape=torch.Size([1, 1404]))
               

In [9]:
print("Run observed simulation")
data = tvb_inference.run_sim(2.2 * np.ones(1))

Run observed simulation
Using params: [2.2]
   INFO  White noise configured with dt=0.005


In [10]:
print("Run Posterior")
post_samples = tvb_inference.posterior(data=data)

Run Posterior
FC_Corr


Generating samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:05<00:00, 15.37it/s]


In [11]:
print(post_samples.mean())

1.7462754


In [12]:
# print("G value found: {}".format(found_value))

G value found: 2.3394737243652344
